DOI: 10.1126/science.aax4438

In [ ]:
import gzip
import os
import shutil
import tempfile

import scanpy as sc

from pertdata import cache_dir_path
from pertdata.utils import download_file

In [ ]:
name = "NormanWeissman2019_Guille"
dir_path = name
os.makedirs(name=dir_path, exist_ok=True)

Download the raw data:

In [ ]:
urls = [
    "https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE133344&format=file&file=GSE133344%5Fraw%5Fbarcodes%2Etsv%2Egz",
    "https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE133344&format=file&file=GSE133344%5Fraw%5Fcell%5Fidentities%2Ecsv%2Egz",
    "https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE133344&format=file&file=GSE133344%5Fraw%5Fgenes%2Etsv%2Egz",
    "https://ftp.ncbi.nlm.nih.gov/geo/series/GSE133nnn/GSE133344/suppl/GSE133344%5Fraw%5Fmatrix%2Emtx%2Egz",
]

filenames = [
    "raw_barcodes.tsv.gz",
    "raw_cell_identities.csv.gz",
    "raw_genes.tsv.gz",
    "raw_matrix.mtx.gz",
]

for url, filename in zip(urls, filenames):
    download_file(url=url, path=os.path.join(dir_path, filename))

We need to rename `raw_genes.tsv.gz` to `raw_features.tsv.gz`, because the function `scanpy.read_10x_mtx` expects the file to be named `raw_features.tsv.gz`.
We make a copy and keep `raw_genes.tsv.gz` to avoid duplicate downloads.

In [ ]:
shutil.copy2(
    src=os.path.join(dir_path, "raw_genes.tsv.gz"),
    dst=os.path.join(dir_path, "raw_features.tsv.gz"),
)

Also, the `raw_features.tsv.gz` file needs to have a third column with the value `Gene Expression`.

In [ ]:
raw_features_file_path = os.path.join(dir_path, "raw_features.tsv.gz")

with gzip.open(filename=raw_features_file_path, mode="rt") as input_file:
    lines = input_file.readlines()
    if not lines[0].strip().endswith("Gene Expression"):
        with tempfile.NamedTemporaryFile() as temp_file:
            temp_file_path = temp_file.name
            with gzip.open(filename=temp_file_path, mode="wt") as output_file:
                for line in lines:
                    line = line.strip() + "\tGene Expression\n"
                    output_file.write(line)
            shutil.copy2(src=temp_file_path, dst=raw_features_file_path)

Load the data:

In [ ]:
adata = sc.read_10x_mtx(path=dir_path, var_names="gene_ids", cache=False, prefix="raw_")

Save the data to an H5AD file:

In [ ]:
h5ad_file_path = os.path.join(dir_path, "adata.h5ad")
adata.write(filename=h5ad_file_path)

Copy the H5AD file to the cache directory:

In [ ]:
dataset_dir_path = os.path.join(cache_dir_path(), name)
os.makedirs(name=dataset_dir_path, exist_ok=True)
shutil.copy2(
    src=h5ad_file_path,
    dst=os.path.join(dataset_dir_path, "adata.h5ad"),
)